In [2]:
import pandas as pd


In [3]:
df = pd.read_csv("Books_df.csv")

In [4]:
df.columns = df.columns.str.replace(" ","_").str.lower().str.replace("no.","num").str.replace(":","")

In [5]:
df

,unnamed_0,title,author,main_genre,sub_genre,type,price,rating,num_of_people_rated,urls
0,0,The Complete Novel of Sherlock Holmes,Arthur Conan Doyle,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹169.00,4.4,19923.0,https://www.amazon.in/Complete-Novels-Sherlock...
1,1,Black Holes (L) : The Reith Lectures [Paperbac...,Stephen Hawking,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹99.00,4.5,7686.0,https://www.amazon.in/Black-Holes-Lectures-Ste...
2,2,The Kite Runner,Khaled Hosseini,"Arts, Film & Photography",Cinema & Broadcast,Kindle Edition,₹175.75,4.6,50016.0,https://www.amazon.in/Kite-Runner-Khaled-Hosse...
3,3,Greenlights: Raucous stories and outlaw wisdom...,Matthew McConaughey,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹389.00,4.6,32040.0,https://www.amazon.in/Greenlights-Raucous-stor...
4,4,The Science of Storytelling: Why Stories Make ...,Will Storr,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹348.16,4.5,1707.0,https://www.amazon.in/Science-Storytelling-Wil...
...,...,...,...,...,...,...,...,...,...,...
7923,7923,Insight Guides Poland (Travel Guide with Free ...,Insight Travel Guide,Travel,Travel & Holiday Guides,Paperback,"₹1,326.00",4.7,16.0,https://www.amazon.in/Insight-Guides-Poland-Tr...
7924,7924,Lonely Planet India 19 (Travel Guide),Anirban Mahapatra,Travel,Travel & Holiday Guides,Paperback,₹850.00,4.4,187.0,https://www.amazon.in/Lonely-Planet-India-Trav...
7925,7925,Eyewitness Travel Phrase Book French (EW Trave...,DK,Travel,Travel & Holiday Guides,Paperback,₹307.00,4.5,168.0,https://www.amazon.in/Eyewitness-Travel-Phrase...
7926,7926,Lonely Planet Australia (Travel Guide),Andrew Bain,Travel,Travel & Holiday Guides,Kindle Edition,"₹1,814.50",4.7,267.0,https://www.amazon.in/Lonely-Planet-Australia-...


In [90]:
df.dtypes

unnamed_0                int64
title                   object
author                  object
main_genre              object
sub_genre               object
type                    object
price                   object
rating                 float64
num_of_people_rated    float64
urls                    object
dtype: object

In [93]:
df['main_genre'].value_counts()

main_genre
Children's Books                         1450
Romance                                  1124
Sports                                    854
Literature & Fiction                      400
Computing, Internet & Digital Media       350
Teen & Young Adult                        350
Religion                                  300
Crafts, Home & Lifestyle                  300
Medicine & Health Sciences                250
Arts, Film & Photography                  200
Society & Social Sciences                 200
Reference                                 200
Higher Education Textbooks                200
Exam Preparation                          200
Sciences, Technology & Medicine           200
Politics                                  150
Business & Economics                      150
Science & Mathematics                     150
History                                   100
Health, Family & Personal Development     100
Biographies, Diaries & True Accounts      100
Fantasy, Horror & Scien

In [7]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests
def titulos_con_similitud():
   def buscar_libros_por_titulo_con_similitud(titulo):
    
    base_url = "https://openlibrary.org/search.json"
    params = {'title': titulo}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        libros = data.get('docs', [])
        resultados = []
        for libro in libros:
            resultados.append({
            'titulo': libro.get('title'),
            'autor': libro.get('author_name', ['Desconocido'])[0],
            'isbn': libro.get('isbn', [None])[0],
            'año_publicacion': libro.get('publish_year', [None])[0]
            })
        
        return resultados
    else:
        return f"Error en la búsqueda: código de estado {response.status_code}"
   for i in range (0,len(df)):
    try:
        autor = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['autor']
        df.loc[i, 'autor_api'] = autor
    
        year_public = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['año_publicacion']
        df.loc[i, 'year_public'] = year_public

        isbn = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['isbn']
        df.loc[i, 'isbn'] = isbn
    except:
        None 

    
def calcular_similitud(texto1, texto2):
    vectorizer = TfidfVectorizer()
    
    # Transforma los textos en vectores TF-IDF
    tfidf = vectorizer.fit_transform([texto1, texto2])
    
    # Calcula la similitud del coseno entre los dos vectores
    similitud = cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]
    return similitud

def buscar_libros_por_titulo_con_similitud(titulo):
    
    base_url = "https://openlibrary.org/search.json"
    params = {'title': titulo}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        libros = data.get('docs', [])
        resultados = []
        for libro in libros:
            resultados.append({
            'titulo': libro.get('title'),
            'autor': libro.get('author_name', ['Desconocido'])[0],
            'isbn': libro.get('isbn', [None])[0],
            'año_publicacion': libro.get('publish_year', [None])[0]
            })
        
        return resultados
    else:
        return f"Error en la búsqueda: código de estado {response.status_code}"

    resultados = buscar_libros_por_titulo(titulo)
    resultados_con_similitud = []

    for libro in resultados:
        similitud = calcular_similitud(titulo, libro['titulo'])
        if similitud >= 0.80:
            libro['similitud'] = similitud
            resultados_con_similitud.append(libro)

    return resultados_con_similitud

In [94]:
df.head()

,unnamed_0,title,author,main_genre,sub_genre,type,price,rating,num_of_people_rated,urls
0,0,The Complete Novel of Sherlock Holmes,Arthur Conan Doyle,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹169.00,4.4,19923.0,https://www.amazon.in/Complete-Novels-Sherlock...
1,1,Black Holes (L) : The Reith Lectures [Paperbac...,Stephen Hawking,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹99.00,4.5,7686.0,https://www.amazon.in/Black-Holes-Lectures-Ste...
2,2,The Kite Runner,Khaled Hosseini,"Arts, Film & Photography",Cinema & Broadcast,Kindle Edition,₹175.75,4.6,50016.0,https://www.amazon.in/Kite-Runner-Khaled-Hosse...
3,3,Greenlights: Raucous stories and outlaw wisdom...,Matthew McConaughey,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹389.00,4.6,32040.0,https://www.amazon.in/Greenlights-Raucous-stor...
4,4,The Science of Storytelling: Why Stories Make ...,Will Storr,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹348.16,4.5,1707.0,https://www.amazon.in/Science-Storytelling-Wil...


In [10]:

for i in range (0,len(df)):
    try:
        autor = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['autor']
        df.loc[i, 'autor_api'] = autor
    
        year_public = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['año_publicacion']
        df.loc[i, 'year_public'] = year_public

        isbn = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['isbn']
        df.loc[i, 'isbn'] = isbn
    except:
        None


In [108]:
df

,unnamed_0,title,author,main_genre,sub_genre,type,price,rating,num_of_people_rated,urls,autor_api,year_public,isbn
0,0,The Complete Novel of Sherlock Holmes,Arthur Conan Doyle,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹169.00,4.4,19923.0,https://www.amazon.in/Complete-Novels-Sherlock...,Arthur Conan Doyle,2017.0,8175994312
1,1,Black Holes (L) : The Reith Lectures [Paperbac...,Stephen Hawking,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹99.00,4.5,7686.0,https://www.amazon.in/Black-Holes-Lectures-Ste...,NaN,NaN,NaN
2,2,The Kite Runner,Khaled Hosseini,"Arts, Film & Photography",Cinema & Broadcast,Kindle Edition,₹175.75,4.6,50016.0,https://www.amazon.in/Kite-Runner-Khaled-Hosse...,Khaled Hosseini,2016.0,0756948800
3,3,Greenlights: Raucous stories and outlaw wisdom...,Matthew McConaughey,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹389.00,4.6,32040.0,https://www.amazon.in/Greenlights-Raucous-stor...,NaN,NaN,NaN
4,4,The Science of Storytelling: Why Stories Make ...,Will Storr,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹348.16,4.5,1707.0,https://www.amazon.in/Science-Storytelling-Wil...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7923,7923,Insight Guides Poland (Travel Guide with Free ...,Insight Travel Guide,Travel,Travel & Holiday Guides,Paperback,"₹1,326.00",4.7,16.0,https://www.amazon.in/Insight-Guides-Poland-Tr...,NaN,NaN,NaN
7924,7924,Lonely Planet India 19 (Travel Guide),Anirban Mahapatra,Travel,Travel & Holiday Guides,Paperback,₹850.00,4.4,187.0,https://www.amazon.in/Lonely-Planet-India-Trav...,NaN,NaN,NaN
7925,7925,Eyewitness Travel Phrase Book French (EW Trave...,DK,Travel,Travel & Holiday Guides,Paperback,₹307.00,4.5,168.0,https://www.amazon.in/Eyewitness-Travel-Phrase...,NaN,NaN,NaN
7926,7926,Lonely Planet Australia (Travel Guide),Andrew Bain,Travel,Travel & Holiday Guides,Kindle Edition,"₹1,814.50",4.7,267.0,https://www.amazon.in/Lonely-Planet-Australia-...,NaN,NaN,NaN


In [37]:
df.isnull()

,unnamed_0,title,author,main_genre,sub_genre,type,price,rating,num_of_people_rated,urls,autor_api,year_public,isbn
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,True,True,True
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,True,True,True
4,False,False,False,False,False,False,False,False,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7923,False,False,False,False,False,False,False,False,False,False,True,True,True
7924,False,False,False,False,False,False,False,False,False,False,True,True,True
7925,False,False,False,False,False,False,False,False,False,False,True,True,True
7926,False,False,False,False,False,False,False,False,False,False,True,True,True


In [35]:
df.dropna(subset=["autor_api"])

,unnamed_0,title,author,main_genre,sub_genre,type,price,rating,num_of_people_rated,urls,autor_api,year_public,isbn
0,0,The Complete Novel of Sherlock Holmes,Arthur Conan Doyle,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹169.00,4.4,19923.0,https://www.amazon.in/Complete-Novels-Sherlock...,Arthur Conan Doyle,2017.0,8175994312
2,2,The Kite Runner,Khaled Hosseini,"Arts, Film & Photography",Cinema & Broadcast,Kindle Edition,₹175.75,4.6,50016.0,https://www.amazon.in/Kite-Runner-Khaled-Hosse...,Khaled Hosseini,2016.0,0756948800
5,5,Fantastic Beasts and Where to Find Them: The O...,J.K. Rowling,"Arts, Film & Photography",Cinema & Broadcast,Kindle Edition,₹313.95,4.5,15122.0,https://www.amazon.in/Fantastic-Beasts-Where-F...,J. K. Rowling,2016.0,0606396683
9,9,Screenplay,Syd Field,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹644.52,4.5,3216.0,https://www.amazon.in/Screenplay-Foundations-S...,J. K. Rowling,2016.0,0606396683
10,10,One Day,David Nicholls,"Arts, Film & Photography",Cinema & Broadcast,Audible Audiobook,₹892.00,4.2,10550.0,https://www.amazon.in/One-Day/dp/B079P515NK/re...,Arnold Bennett,1925.0,9780341688341
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7461,7461,Lucky Hit,Hannah Cowan,Sports,Winter Sports,Paperback,₹891.00,4.1,6332.0,https://www.amazon.in/Lucky-Hit-Hannah-Cowan/d...,Hannah Cowan,2020.0,9798670001274
7464,7464,Running Wild,K.A. Tucker,Sports,Winter Sports,Paperback,₹341.06,4.5,6233.0,https://www.amazon.in/Running-Wild-Simple-Book...,Alison Fraser,1996.0,9780263795752
7475,7475,Knight of the Ice 11,Yayoi Ogawa,Sports,Winter Sports,Paperback,₹277.63,4.5,18.0,https://www.amazon.in/Knight-Ice-11-Yayoi-Ogaw...,Yayoi Ogawa,2022.0,9781646510887
7479,7479,The People of the Indus,Nikhil Gulati,Teen & Young Adult,Biographies,Hardcover,₹407.00,4.4,250.0,https://www.amazon.in/People-Indus-Jonathan-Ma...,Nikhil Gulati,2022.0,014345532X
